In [ ]:
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import os
from skimage import color

DATADIR = "data/"
fnames = os.listdir(DATADIR)
fnames = [os.path.join(DATADIR, i) for i in fnames]
print(fnames)

In [ ]:
cam1_1 = scipy.io.loadmat(fnames[0])
cam1_1.keys()

In [ ]:
print(cam1_1.shape)
print(cam1_1[:,:,:,0].shape)
y, x, colors, samples = cam1_1.shape

In [ ]:
def greyscale(four_dim_array):
    """
    Assumes the input is [y, x, RGB, sample], reduces to
    [y, x, sample]
    """
    y, x, _, samples = four_dim_array.shape
    output = np.ndarray((y, x, samples))
    f = plt.figure(1)
    p = f.add_subplot(111)
    image = p.imshow(four_dim_array[:, :, :, 0])
    for i in range(1, samples):
        tmp = four_dim_array[:, :, :, i]
        image.set_data(tmp)
        plt.pause(0.02)
       

In [ ]:
greyscale(cam1_1)